In [ ]:
import os
import json
import shutil
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

In [ ]:
index2image = []
image2index = {}
with open('./index.txt') as f:
    lines = f.readlines()
for line in lines:
    index = line.split(' ')[0]
    image = line.split(' ')[1].split('#')[1]
    index2image.append(image)
    image2index[image] = index
def decoder(picture_name):
    return image2index[picture_name.split('#')[1]]

In [ ]:
ground_truth = pd.read_excel('./label.xlsx')
label = []
for index, row in ground_truth.iterrows():
    objects_list = row['objects']
    if len(objects_list.split('&')[0]) > 0:
        label.append(objects_list.split('&')[0].split('#'))
    else:
        label.append([])
print(label)

In [ ]:
resolution = 30
result_folder = './detr_video_result/' + str(resolution)
test_truth = []
test_result = []
def get_count(object_list, object_label):
    count = 0
    for object in object_list:
        if object == object_label:
            count += 1
    return count
for objects_file in sorted(os.listdir(result_folder)):
    with open(os.path.join(result_folder, objects_file), 'r') as f:
        objects_info = json.load(f)
    objects_list = []
    temp_result = 0
    for objects_dict in objects_info['objects']:
        objects_list.append(objects_dict['label'])
    picture_index = (int)(decoder(objects_file))
    label_final = np.asarray(label[picture_index])
    object_final = np.asarray(objects_list)
    for object_label in label_final:
        temp_result += get_count(object_final, object_label) >= get_count(label_final, object_label)
    test_truth.append(len(label_final))
    test_result.append(temp_result >= 1)
report = classification_report(test_truth, test_result)
print(report)
print(test_result, test_truth)